In [ ]:
# default_exp audio.mixup

# MixUp for Audio

> Apply MixUp to audio waveforms before converting to Spectrogram or MelSpectrogram

In [ ]:
#export
from __future__ import annotations

from fastcore.transform import Pipeline

from fastai.callback.mixup import MixUp

from fastxtend.audio.data import MelSpectrogram, Spectrogram
from fastxtend.imports import *

In [ ]:
#export
class AudioMixUp(MixUp):
    "Implementation of https://arxiv.org/abs/1710.09412 on audio waveforms"

    def before_fit(self):
        waveforms, wave, spec = True, [], []
        self._wave_pipe = Pipeline([])
        self._spec_pipe = Pipeline([])
        
        # first copy transforms
        self._orig_pipe = self.dls.train.after_batch
        
        # loop through existing transforms appending to pre_spec/post_spec until Spec/Mel is found
        for i in range(len(self.dls.train.after_batch.fs)):
            if isinstance(self.dls.train.after_batch[i], (Spectrogram, MelSpectrogram)):
                waveforms = False
            if waveforms: wave.append(self.dls.train.after_batch[i])
            else:         spec.append(self.dls.train.after_batch[i])

        self._wave_pipe.add(wave)
        self._spec_pipe.add(spec)

        # set existing transforms to an empty Pipeline
        self.dls.train.after_batch = Pipeline([])

    def before_batch(self):
        self.learn.xb = self._wave_pipe(self.xb)
        super().before_batch()
        self.learn.xb = self._spec_pipe(self.xb)

    def after_fit(self):
        self.dls.train.after_batch = self._orig_pipe

    def after_cancel_fit(self):
        self.after_fit()
        super().after_cancel_fit()